# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,6.56,95,100,17.92,TF,1696125747
1,1,badger,64.8000,-147.5333,10.14,47,0,3.09,US,1696125612
2,2,daru,-9.0763,143.2092,26.12,81,100,6.99,PG,1696125747
3,3,margaret river,-33.9500,115.0667,16.24,72,96,9.42,AU,1696125748
4,4,qaqortoq,60.7167,-46.0333,2.17,85,70,0.33,GL,1696125748


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600,
    title = f"Humidity Per City (10/2/23)")

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
df = city_data_df[(city_data_df["Max Temp"] >= 25) & (city_data_df["Max Temp"] < 50) & 
                        (city_data_df["Cloudiness"] >= 10) & (city_data_df["Cloudiness"] < 65) &
                        (city_data_df["Wind Speed"] < 5)]

# Drop any rows with null values
df = df.dropna(how="any")

# Display sample data
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,ambon,-3.6954,128.1814,29.08,79,40,3.09,ID,1696125751
33,33,itamaraca,-7.7478,-34.8256,25.03,79,31,4.05,BR,1696125753
49,49,dili,-8.5586,125.5736,28.57,62,50,2.80,TL,1696125755
62,62,holualoa,19.6228,-155.9522,26.97,80,42,1.34,US,1696125758
64,64,wailua homesteads,22.0669,-159.3780,30.24,77,30,1.79,US,1696125758


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
20,ambon,ID,-3.6954,128.1814,79,
33,itamaraca,BR,-7.7478,-34.8256,79,
49,dili,TL,-8.5586,125.5736,62,
62,holualoa,US,19.6228,-155.9522,80,
64,wailua homesteads,US,22.0669,-159.3780,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
     "apiKey" : geoapify_key,
    "limit" : 20,
    "categories" : "accommodation.hotel",
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ambon - nearest hotel: Hotel Hero
itamaraca - nearest hotel: No hotel found
dili - nearest hotel: Hotel Timor
holualoa - nearest hotel: Kona Hotel
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
georgetown - nearest hotel: Page 63 hostel
hawaiian paradise park - nearest hotel: No hotel found
padang - nearest hotel: losmen Surya
limon - nearest hotel: Hotel NG
kone - nearest hotel: Pacifik Appartels
coahuayana de hidalgo - nearest hotel: No hotel found
mahina - nearest hotel: Motu Martin
ad dabbah - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: Hotel del Valle
kavieng - nearest hotel: Kavieng Hotel
benjamin constant - nearest hotel: No hotel found
vavuniya - nearest hotel: Vanniyan Hotel
paka - nearest hotel: Paka Inn
puerto lempira - nearest hotel: Hotel Samaritano
ixtapa - nearest hotel: Ma. Cristina
naze - nearest hotel: ビジネス旅館畠山
cidade velha - nearest hotel: Encosta Azul
una - nearest hotel: No hotel found
carutapera - 

,City,Country,Lat,Lng,Humidity,Hotel Name
20,ambon,ID,-3.6954,128.1814,79,Hotel Hero
33,itamaraca,BR,-7.7478,-34.8256,79,No hotel found
49,dili,TL,-8.5586,125.5736,62,Hotel Timor
62,holualoa,US,19.6228,-155.9522,80,Kona Hotel
64,wailua homesteads,US,22.0669,-159.3780,77,Hilton Garden Inn Kauai Wailua Bay
78,georgetown,MY,5.4112,100.3354,86,Page 63 hostel
79,hawaiian paradise park,US,19.5933,-154.9731,76,No hotel found
102,padang,ID,-0.9492,100.3543,77,losmen Surya
123,limon,CR,10.0000,-83.0333,82,Hotel NG
125,kone,NC,-21.0595,164.8658,47,Pacifik Appartels


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

map2 = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "EsriNatGeo", 
    color= "City", 
    line_color="black", 
    size=80, 
    frame_width=800, 
    frame_height=600, 
    hover_cols = ["Humidity","Hotel Name", "Country" ],
    title=f"Hotels within 10,000 meters (10/2/23)"
)

# Display the map
map2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)